# Training

In [ ]:
import numpy as np
import cv2
import random
from random import randint
from scipy import ndimage
import os
import sys

from google.colab.patches import cv2_imshow

from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.layers import Dense, Dropout, Flatten, Input, BatchNormalization, Conv2D, MaxPooling2D
from keras.models import Model
from keras import metrics
import keras.backend as backend

from sklearn.metrics import accuracy_score

from tqdm import tqdm

In [ ]:
!mkdir /content/images
!mkdir /content/images/train
!mkdir /content/labels
!mkdir /content/labels/train
!mkdir /content/rotations
!mkdir /content/rotations/train

In [ ]:
# Here are classes named for training
Dict = {0: 'advance_to_contact', 1: 'ambush', 2: 'attack', 3: 'attack_by_fire', 4: 'block',
        5: 'breach', 6: 'clear', 7: 'contain', 8: 'control', 9: 'counterattack', 10: 'cover',
        11: 'delay', 12: 'deny', 13: 'destroy', 14: 'disrupt', 15: 'fix', 16: 'guard',
        17: 'isolate', 18: 'main_attack', 19: 'neutralize', 20: 'occupy', 21: 'penetrate',
        22: 'retain', 23: 'retire', 24: 'screen', 25: 'secure', 26: 'seize', 27: 'support_by_fire',
        28: 'suppress', 29: 'turn', 30: 'withdraw'}

In [ ]:
Dict_images = {key: [] for key in Dict.values()}
Dict_rotations = {key: [] for key in Dict.values()}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip -q '/content/drive/My Drive/NATO/train.zip' -d '/content/images/train'
!unzip -q '/content/drive/My Drive/NATO/train_l.zip' -d '/content/labels/train'
!unzip -q '/content/drive/My Drive/NATO/train_r.zip' -d '/content/rotations/train'

In [ ]:
images = '/content/images/train'
labels = '/content/labels/train'
rotations = '/content/rotations/train'

file_list = os.listdir(images)

for name in tqdm(file_list):
    tmp_list_class = []
    image = cv2.imread(images+"/"+name, cv2.IMREAD_GRAYSCALE)
    with open(labels+"/"+str(name).split(".")[0]+".txt") as f:
        lines = [line.rstrip('\n') for line in f]
        for line in lines:
            x = int((float(line.split(" ")[1])-(float(line.split(" ")[3])/2)) * 770)
            y = int((float(line.split(" ")[2])-(float(line.split(" ")[4])/2)) * 576)
            w = int(float(line.split(" ")[3]) * 770 + 5)
            h = int(float(line.split(" ")[4]) * 576 + 5)

            crop_image = image[y:y+h, x:x+w]
            tmp_list_class.append(int(line.split(" ")[0]))
            dim = (75, 75)
            resized_img = cv2.resize(crop_image, dim, interpolation = cv2.INTER_AREA)
            Dict_images[Dict[int(line.split(" ")[0])]].append(resized_img)
    with open(rotations+"/"+str(name).split(".")[0]+".txt") as a:
        r = [line.rstrip('\n') for line in a]
        i = 0
        for rotation in r:
            Dict_rotations[Dict[tmp_list_class[i]]].append(rotation)
            i += 1

In [ ]:
for i in Dict_images.keys():
    print(str(i))
    cv2_imshow(Dict_images[str(i)][0])

In [ ]:
def angle_error_metric(y_true, y_pred):
    diff = 180 - abs(abs(backend.argmax(y_true) - backend.argmax(y_pred)) - 180)
    return backend.mean(backend.cast(backend.abs(diff), backend.floatx()))

def angle_error(y_true, y_pred):
    diff = 180 - abs(abs(y_true - y_pred) - 180)
    return np.abs(diff)

In [ ]:
for i in Dict_images.keys():
    print(str(i))
    x = Dict_images[str(i)]
    y = Dict_rotations[str(i)]

    y_categorical = to_categorical(y, 360)

    x_train = np.array(x[0:])
    y_train = np.array(y_categorical)

    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)

    print(x_train.shape, y_train.shape)

    # number of classes
    classes = 360

    # model
    #input = Input(shape=(75, 75, 1))
    #x = Conv2D(64, 3, 3, activation='relu')(input)
    #x = Dense(256, activation='relu')(x)
    #x = BatchNormalization()(x)
    #x = Conv2D(64, 3, 3, activation='relu')(x)
    #x = Dense(128, activation='relu')(x)
    #x = BatchNormalization()(x)
    #x = Flatten()(x)
    #x = Dense(classes, activation='softmax')(x)

    input = Input(shape=(75, 75, 1))
    x = Conv2D(64, 3, 3, padding = 'same', activation='relu')(input)
    x = Conv2D(64, 3, 3, padding = 'same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(64, 3, 3, padding = 'same', activation='relu')(x)
    x = Conv2D(64, 3, 3, padding = 'same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2), padding = 'same')(x)
    x = Dropout(0.25)(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dense(classes, activation='softmax')(x)

    model = Model(inputs=input, outputs=x)

    # Compile the model
    model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=[metrics.categorical_accuracy, angle_error_metric])

    # Fit the model
    model.fit(x_train, y_train, batch_size = 64, epochs = 500)

    model.save(str(i)+"_rotation_model.h5")

In [ ]:
import shutil

for i in Dict_images.keys():
    colab_link = '/content/'+str(i)+'_rotation_model.h5'
    gdrive_link = "/content/drive/My Drive/models_rotation/"
    shutil.copy(colab_link, gdrive_link)

# Prediction

In [ ]:
!mkdir /content/images
!mkdir /content/images/test
!mkdir /content/labels
!mkdir /content/labels/test
!mkdir /content/rotations
!mkdir /content/rotations/test

In [ ]:
!unzip -q '/content/drive/My Drive/NATO/test.zip' -d '/content/images/test'
!unzip -q '/content/drive/My Drive/NATO/test_l.zip' -d '/content/labels/test'
!unzip -q '/content/drive/My Drive/NATO/test_r.zip' -d '/content/rotations/test'

In [ ]:
Dict_images_test = {key: [] for key in Dict.values()}
Dict_rotations_test = {key: [] for key in Dict.values()}
Dict_predicted_test = {key: [] for key in Dict.values()}
Dict_angle_errors_test  = {key: [] for key in Dict.values()}

In [ ]:
classes = 360

input = Input(shape=(75, 75, 1))
x = Conv2D(64, 3, 3, padding = 'same', activation='relu')(input)
x = Conv2D(64, 3, 3, padding = 'same', activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Conv2D(64, 3, 3, padding = 'same', activation='relu')(x)
x = Conv2D(64, 3, 3, padding = 'same', activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2), padding = 'same')(x)
x = Dropout(0.25)(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(classes, activation='softmax')(x)

model = Model(inputs=input, outputs=x)

In [ ]:
images_test = '/content/images/test'
labels_test = '/content/labels/test'
rotations_test = '/content/rotations/test'

file_list = os.listdir(images_test)

for name in tqdm(file_list):
    tmp_list_class = []
    image = cv2.imread(images_test+"/"+name, cv2.IMREAD_GRAYSCALE)
    with open(labels_test+"/"+str(name).split(".")[0]+".txt") as f:
        lines = [line.rstrip('\n') for line in f]
        for line in lines:
            x = int((float(line.split(" ")[1])-(float(line.split(" ")[3])/2)) * 770)
            y = int((float(line.split(" ")[2])-(float(line.split(" ")[4])/2)) * 576)
            w = int(float(line.split(" ")[3]) * 770 + 5)
            h = int(float(line.split(" ")[4]) * 576 + 5)

            crop_image = image[y:y+h, x:x+w]
            tmp_list_class.append(int(line.split(" ")[0]))
            dim = (75, 75)
            resized_img = cv2.resize(crop_image, dim, interpolation = cv2.INTER_AREA)
            Dict_images_test[Dict[int(line.split(" ")[0])]].append(resized_img)

            ## Predict
            reshaped_img = resized_img.reshape(1, 75, 75)
            model.load_weights('/content/drive/MyDrive/models_rotation/'+Dict[int(line.split(" ")[0])]+'_rotation_model_31.h5')
            predicted_rotation = model.predict(reshaped_img, verbose=0)
            Dict_predicted_test[Dict[int(line.split(" ")[0])]].append(np.argmax(predicted_rotation))
    with open(rotations_test+"/"+str(name).split(".")[0]+".txt") as a:
        r = [line.rstrip('\n') for line in a]
        i = 0
        for rotation in r:
            Dict_rotations_test[Dict[tmp_list_class[i]]].append(rotation)
            i += 1

In [ ]:
for i in Dict_images.keys():
    print(str(i))
    symbol_name = str(i)

    print(len(Dict_rotations_test[symbol_name]))
    for i in range(len(Dict_rotations_test[symbol_name])):
        Dict_angle_errors_test[symbol_name].append(angle_error(int(Dict_rotations_test[symbol_name][i]), int(Dict_predicted_test[symbol_name][i])))

    print(len(Dict_angle_errors_test[symbol_name]), round(np.mean(Dict_angle_errors_test[symbol_name]), 2))

    kokku = sum(i <= 30 for i in Dict_angle_errors_test[symbol_name])
    print(kokku, str(round(kokku/len(Dict_angle_errors_test[symbol_name])*100, 2))+"%")

In [ ]:
symbol_name = "advance_to_contact"

res = tuple(zip(Dict_angle_errors_test[symbol_name], Dict_rotations_test[symbol_name], Dict_predicted_test[symbol_name]))
sorted(res, key = lambda x: (-x[0],x[1],x[2]))